# Data extraction

## Some sources to extract data


###  1.-  Dayli data from all coins available at: https://coinmarketcap.com/coins/ with the function below
###  2.-  Hourly and daily data in csv format from some coins abailable at: https://www.cryptodatadownload.com/data/ 

In [1]:
import pandas as pd

In [22]:
# scrapping first 200 coins names available

url = 'https://coinmarketcap.com/coins/views/all/'
dfs = pd.read_html(url)
df = dfs[2]
df.head()

,Rank,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d,Unnamed: 10
0,1,Bitcoin,BTC,"$205,561,895,837","$11,141.68","18,449,812 BTC","$27,096,537,474",0.11%,-5.93%,10.81%,NaN
1,2,Ethereum,ETH,"$42,647,593,778",$380.71,"112,020,368 ETH","$18,935,636,725",-0.11%,-3.31%,19.54%,NaN
2,3,XRP,XRP,"$13,234,045,630",$0.294990,"44,862,646,997 XRP *","$4,918,859,452",-0.49%,-1.58%,36.08%,NaN
3,4,Bitcoin Cash,BCH,"$5,328,532,447",$288.35,"18,479,206 BCH","$3,318,026,091",-0.12%,-11.24%,15.51%,NaN
4,5,Bitcoin SV,BSV,"$4,047,694,657",$219.06,"18,477,777 BSV","$2,630,128,220",-0.37%,-11.23%,13.92%,NaN


In [23]:
df.Price = df['Price'].apply(lambda x: float(x.replace('$','').replace(',','')))
df.sort_values(by=['Price'], inplace=True, ascending=False)

In [25]:
import plotly.express as px

fig = px.bar(df, y='Price', x='Name', text='Name')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [26]:
coins_available = list(df.Name)

In [27]:
len(coins_available)

200

In [28]:
# Yes I know there are better ways to do that

coins_available = [(coin.lower()).replace(' ','-').replace('.','-').replace('[','').replace(']','').replace('#','') for coin in coins_available]

In [29]:
print(coins_available)

['bitcoin', 'ethereum', 'bitcoin-cash', 'bitcoin-sv', 'bitcoinpos', 'monero', 'dash', 'zcash', 'unobtanium', 'litecoin', 'bonorum', 'obyte', 'binance-coin', 'counos-x', 'salus', 'mimblewimblecoin', 'decred', 'neo', 'bitcoin-gold', 'kusama', 'largo-coin', 'newyork-exchange', 'energy-web-token', 'horizen', 'ethereum-classic', 'ddkoin', 'zcoin', 'bitcoinhd', 'platincoin', 'counos-coin', 'cosmos', 'heat', 'eos', 'tezos', 'ghostprism', 'qtum', 'elastos', 'zenon', 'chainx', 'btse', 'vitae', 'energi', 'monacoin', 'solana', 'cryptonex', 'celo', 'waves', 'factom', 'haven-protocol', 'lightning-bitcoin', 'hypercash', 'lisk', 'blocknet', 'dero', 'steem-dollars', 'nano', 'tomochain', 'stakecubecoin', 'bitcoin-diamond', 'bitcoin-2', 'baer-chain', 'bhpcoin', 'particl', 'ontology', 'zano', 'polis', 'komodo', 'insolar', 'fusion', 'skycoin', 'deviantcoin', 'loki', 'neblio', 'ark', 'stratis', 'ouroboros', 'aidos-kuneen', 'gxchain', 'grin', 'aeon', 'nuls', 'pivx', 'nebulas', 'beam', 'monero-classic', 'ori

## Function

## Function to obtain some coins at: https://coinmarketcap.com/coins/



In [ ]:
# Libraries
import urllib.request as urllib2
from bs4 import BeautifulSoup
import datetime 
import csv

In [30]:
'''coins_available = ['bitcoin','ethereum','xrp','bitcoin-cash','bitcoin-sv','litecoin','cardano','binance-coin','eos','tezos','stellar','monero','tron','vechain',
                   'neo','ethereum-classic','dash','zcash','iota','cosmos','nem','ontology','dogecoin','digibyte','theta','qtum','algorand','hedera-hashgraph',
                   'icon','decred','zilliqa','bitcoin-gold','waves','bitcoin-diammond','lisk','ravencoin','siacoin','nano','monacoin','terra','bytom','nervos-network',
                   'divi','verge','blockstack','horizen','iost','komodo','steem','abbc-coin','bitshares','zcoin','hive','energi','tomochain','hypercash','ark',
                   'xinfin-network','syscoin','wax','velas','ardor','irisnet','aeternity','v-systems','stratis','elastos','harmony','electroneum','aion','super-zero-protocol',
                   'nimiq','nuls','waykichain','bytecoin','reddcoin','wanchain','gxchain','theta-fuel','iotex','kusama','thunder-token','project-pai','solana','loki',
                   'cortex','beam','fusion','truechain','pivx','grin','insolar','nebulas','coti','counos-x','bitcoinpos','celo','largo-coin','mimblewimblecoin',
                   'advanced-internet-blocks']'''

def get_data(coins_available):

  print(coins_available)

  coin_name = input('Type a valid coin name: ')

  while True:

    if coin_name not in coins_available:
      print('Coin name should not contain spaces instead use "-" and type all in lowecase')
      coin_name = input('Type again a valid coin name: ')
    else:
      break
  
  today = datetime.date.today()
  mktcap_page = 'https://coinmarketcap.com/currencies/'+ coin_name +'/historical-data/?start=20130428&amp;end='+"{:%Y%m%d}".format(today)
  page = urllib2.urlopen(mktcap_page)
  soup = BeautifulSoup(page, 'html.parser')
  table = soup.find("table")
  headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  headings = [x.replace('*','') for x in headings]

  datasets = []

  for row in soup.find_all("tr")[1:]:
      dataset = dict(zip(headings, (tr.get_text() for tr in row.find_all("td"))))
      if len(dataset) > 1:
          datasets.append(dataset)

  my_df = pd.DataFrame(datasets)
  full_data = my_df.set_index('Date')


  # This function replaces ',' for nothing and then changes data type object to float

  def obj_to_num(df,cols):
      
      def obj_to_num(row):
          if ',' in str(row):
            row = row.replace(',','')
            return float(row)
          else:
            return float(row)

      for col in cols:     
          df[col] = df[col].apply(obj_to_num) 
      return df

  # This function changes data column (object) to timestamp[ns] and also can change all dataframe clumns to float 

  def prepare_data(file):

    dateparse = lambda x: datetime.datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d')
    data  = pd.read_csv(file, parse_dates=[0], date_parser=dateparse)
    columns = ['Open','High','Low','Close','Volume','Market Cap']
    num_data = obj_to_num(data,columns)
    data['Date'] = data['Date'].dt.tz_localize(None)
    data = data.groupby([pd.Grouper(key='Date', freq='d')]).first().reset_index()
    data = data.set_index('Date')
    data = data[['High','Low']]
    data.dropna(subset=['High','Low'], inplace=True)

    return data



  return coin_name

print(get_data(coins_available))


['bitcoin', 'ethereum', 'bitcoin-cash', 'bitcoin-sv', 'bitcoinpos', 'monero', 'dash', 'zcash', 'unobtanium', 'litecoin', 'bonorum', 'obyte', 'binance-coin', 'counos-x', 'salus', 'mimblewimblecoin', 'decred', 'neo', 'bitcoin-gold', 'kusama', 'largo-coin', 'newyork-exchange', 'energy-web-token', 'horizen', 'ethereum-classic', 'ddkoin', 'zcoin', 'bitcoinhd', 'platincoin', 'counos-coin', 'cosmos', 'heat', 'eos', 'tezos', 'ghostprism', 'qtum', 'elastos', 'zenon', 'chainx', 'btse', 'vitae', 'energi', 'monacoin', 'solana', 'cryptonex', 'celo', 'waves', 'factom', 'haven-protocol', 'lightning-bitcoin', 'hypercash', 'lisk', 'blocknet', 'dero', 'steem-dollars', 'nano', 'tomochain', 'stakecubecoin', 'bitcoin-diamond', 'bitcoin-2', 'baer-chain', 'bhpcoin', 'particl', 'ontology', 'zano', 'polis', 'komodo', 'insolar', 'fusion', 'skycoin', 'deviantcoin', 'loki', 'neblio', 'ark', 'stratis', 'ouroboros', 'aidos-kuneen', 'gxchain', 'grin', 'aeon', 'nuls', 'pivx', 'nebulas', 'beam', 'monero-classic', 'ori